In [1]:
!rm -r data
!mkdir data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet \
    -O data/yellow_tripdata_2022-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet \
    -O data/yellow_tripdata_2022-02.parquet

--2023-05-20 22:12:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Распознаётся d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)… 52.85.114.54, 52.85.114.180, 52.85.114.39, ...
Подключение к d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.114.54|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 38139949 (36M) [application/x-www-form-urlencoded]
Сохранение в: «data/yellow_tripdata_2022-01.parquet»

data/yellow_tripdat 100%[===================>]  36,37M  9,89MB/s    за 3,7s    

2023-05-20 22:12:21 (9,91 MB/s) - «data/yellow_tripdata_2022-01.parquet» сохранён [38139949/38139949]

--2023-05-20 22:12:21--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Распознаётся d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)… 52.85.114.180, 52.85.114.39, 52.85.114.114, ...
Подключение к d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.

## Question 1

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')
df_valid = pd.read_parquet('data/yellow_tripdata_2022-02.parquet')

df_train.shape, df_valid.shape

((2463931, 19), (2979431, 19))

In [4]:
df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

## Question 2

In [5]:
def calc_duration(df):
    return (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.seconds / 60

df_train['duration'] = calc_duration(df_train)
df_valid['duration'] = calc_duration(df_valid)

In [6]:
df_train['duration'].std()

56.185706892623244

## Question 3

In [7]:
def drop_outliers(df):
    cnt_before = df.shape[0]
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    cnt_after = df.shape[0]
    print(cnt_after / cnt_before)
    return df

df_train = drop_outliers(df_train)
df_valid = drop_outliers(df_valid)

0.9827551989077616
0.9794450685382544


## Question 4

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder

from tqdm.auto import tqdm

In [9]:
cat_features = ['PULocationID', 'DOLocationID']

In [10]:
for col in cat_features:
    df_train[col] = df_train[col].astype('category')
    df_valid[col] = df_valid[col].astype(df_train[col].dtype)

### Dense Solution

In [11]:
def ohe_cat_features(df):
    return pd.get_dummies(df, columns=cat_features)

In [12]:
%%time

r1 = ohe_cat_features(df_train[cat_features + ['duration']])
r1.shape

CPU times: user 5.54 s, sys: 817 ms, total: 6.36 s
Wall time: 6.44 s


(2421441, 516)

In [13]:
r1.head()

,duration,PULocationID_1,PULocationID_2,PULocationID_3,PULocationID_4,PULocationID_5,PULocationID_6,PULocationID_7,PULocationID_8,PULocationID_9,...,DOLocationID_256,DOLocationID_257,DOLocationID_258,DOLocationID_259,DOLocationID_260,DOLocationID_261,DOLocationID_262,DOLocationID_263,DOLocationID_264,DOLocationID_265
0,17.816667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.400000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.966667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.033333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37.533333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def ohe_cat_features(df, vec=None):
    if vec is None:
        vec = OneHotEncoder(sparse=False, dtype='int8')
        vec = vec.fit(df[cat_features])
        
    df_ohe = pd.DataFrame(
        vec.transform(df[cat_features]),
        columns=[
            f'{fname}_{v}'
            for fname, fvals in zip(cat_features, vec.categories_)
            for v in fvals
        ],
        index=df.index,
    )

    df.drop(columns=cat_features, inplace=True)
    df = pd.concat([df, df_ohe], ignore_index=False, copy=False, axis=1)

    return df, vec

In [15]:
%%time

r2, _ = ohe_cat_features(df_train[cat_features + ['duration']])
r2.shape

CPU times: user 924 ms, sys: 364 ms, total: 1.29 s
Wall time: 1.32 s


/var/folders/kb/40bs5mxn3yz55kv3_1ppflwr0000gn/T/ipykernel_27946/3607675724.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cat_features, inplace=True)


(2421441, 516)

In [16]:
r2.head()

,duration,PULocationID_1,PULocationID_2,PULocationID_3,PULocationID_4,PULocationID_5,PULocationID_6,PULocationID_7,PULocationID_8,PULocationID_9,...,DOLocationID_256,DOLocationID_257,DOLocationID_258,DOLocationID_259,DOLocationID_260,DOLocationID_261,DOLocationID_262,DOLocationID_263,DOLocationID_264,DOLocationID_265
0,17.816667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.400000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.966667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.033333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37.533333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
def ohe_cat_features(df, vec=None):
    if vec is None:
        vec = OneHotEncoder(sparse=False, dtype='int8')
        vec = vec.fit(df[cat_features])
        
    df_ohe = pd.DataFrame(
        vec.transform(df[cat_features]),
        columns=[
            f'{fname}_{v}'
            for fname, fvals in zip(cat_features, vec.categories_)
            for v in fvals
        ],
        index=df.index,
    )

    df.drop(columns=cat_features, inplace=True)
    df = pd.concat([df, df_ohe], ignore_index=False, copy=False, axis=1)

    return df, vec

In [18]:
assert (r1.columns.sort_values() == r2.columns.sort_values()).all()

for col in tqdm(r1.columns):
    assert np.allclose(r1[col], r2[col])

  0%|          | 0/516 [00:00<?, ?it/s]

In [19]:
def ohe_cat_features(df, vec=None):
    for col in cat_features:
        df[col] = df[col].map(str)
    
    if vec is None:
        vec = DictVectorizer(sparse=False)

        df = pd.DataFrame(
            vec.fit_transform(df.to_dict(orient='records')),
            columns=vec.feature_names_,
            index=df.index,
        )
    else:
        df = pd.DataFrame(
            vec.transform(df.to_dict(orient='records')),
            columns=vec.feature_names_,
            index=df.index,
        )
    
    # FIXME: this unappropirate
    # ohe_columns = df.columns[df.columns.str.contains('=')]
    # for col in tqdm(ohe_columns):
    #     df[col] = (df[col] > 0).astype(int)
    df.columns = df.columns.str.replace('=', '_')

    return df, vec

In [20]:
%%time

r3, vec = ohe_cat_features(df_train[cat_features + ['duration']])
r3.shape

/var/folders/kb/40bs5mxn3yz55kv3_1ppflwr0000gn/T/ipykernel_27946/2417098221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(str)
/var/folders/kb/40bs5mxn3yz55kv3_1ppflwr0000gn/T/ipykernel_27946/2417098221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(str)


CPU times: user 14.7 s, sys: 4.95 s, total: 19.7 s
Wall time: 21.2 s


(2421441, 516)

In [21]:
r3.head()

,DOLocationID_1,DOLocationID_10,DOLocationID_100,DOLocationID_101,DOLocationID_102,DOLocationID_105,DOLocationID_106,DOLocationID_107,DOLocationID_108,DOLocationID_109,...,PULocationID_90,PULocationID_91,PULocationID_92,PULocationID_93,PULocationID_94,PULocationID_95,PULocationID_96,PULocationID_97,PULocationID_98,duration
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.816667
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.400000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.966667
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.033333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.533333


Using `OneHotEncoder` is the best solution.

### Sparse Solution

In [22]:
import scipy
import scipy.sparse as sp

In [23]:
def ohe_cat_features_sparse(df):
    return sp.hstack([
        sp.csr_matrix(df.drop(columns=cat_features).values),
        pd.get_dummies(df[cat_features], columns=cat_features, sparse=True).sparse.to_coo(),
    ], format='csr')

In [24]:
%%time

r1 = ohe_cat_features_sparse(df_train[cat_features + ['duration']])
r1.shape

CPU times: user 2.32 s, sys: 338 ms, total: 2.66 s
Wall time: 2.76 s


(2421441, 516)

In [25]:
def ohe_cat_features_sparse(df, vec=None):
    if vec is None:
        vec = OneHotEncoder(sparse=True, dtype='int8')
    
        df = sp.hstack([
            sp.csr_matrix(df.drop(columns=cat_features).values),
            vec.fit_transform(df[cat_features]),
        ], format='csr')
    else:
        df = sp.hstack([
            sp.csr_matrix(df.drop(columns=cat_features).values),
            vec.transform(df[cat_features]),
        ], format='csr')

    return df, vec

In [26]:
%%time

r2, _ = ohe_cat_features_sparse(df_train[cat_features + ['duration']])
r2.shape

CPU times: user 857 ms, sys: 101 ms, total: 958 ms
Wall time: 976 ms


(2421441, 516)

In [27]:
def ohe_cat_features_sparse(df, vec=None):
    for col in cat_features:
        df[col] = df[col].map(str)
    
    if vec is None:
        vec = DictVectorizer(sparse=True)
        df = vec.fit_transform(df.to_dict(orient='records'))
    else:
        df = vec.transform(df.to_dict(orient='records'))

    return df, vec

In [28]:
%%time

r3, _ = ohe_cat_features_sparse(df_train[cat_features + ['duration']])
r3.shape

/var/folders/kb/40bs5mxn3yz55kv3_1ppflwr0000gn/T/ipykernel_27946/2509363539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(str)
/var/folders/kb/40bs5mxn3yz55kv3_1ppflwr0000gn/T/ipykernel_27946/2509363539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(str)


CPU times: user 13 s, sys: 920 ms, total: 13.9 s
Wall time: 14.2 s


(2421441, 516)

`OneHotEncoder` is the winner here too.

## Question 5

In [29]:
def ohe_cat_features_sparse(df, vec=None):
    if vec is None:
        vec = OneHotEncoder(sparse=True, handle_unknown='ignore', dtype='int8')
    
        df = sp.hstack([
            sp.csr_matrix(df.drop(columns=cat_features).values),
            vec.fit_transform(df[cat_features]),
        ], format='csr')
    else:
        df = sp.hstack([
            sp.csr_matrix(df.drop(columns=cat_features).values),
            vec.transform(df[cat_features]),
        ], format='csr')

    return df, vec

In [30]:
X_train, vec = ohe_cat_features_sparse(df_train[cat_features])
y_train = df_train['duration']

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [32]:
%%time

model = LinearRegression()
model = model.fit(X_train, y_train)

CPU times: user 1min 9s, sys: 14.3 s, total: 1min 23s
Wall time: 21.6 s


In [33]:
mean_squared_error(y_train, model.predict(X_train), squared=False)

6.986191909616623

### Question 6

In [34]:
X_valid, vec = ohe_cat_features_sparse(df_valid[cat_features], vec=vec)
y_valid = df_valid['duration']

In [35]:
mean_squared_error(y_valid, model.predict(X_valid), squared=False)

7.786424875685275